In [1]:
from pathlib import Path
cal_py = r'''
# calibrate_drift_cmdvel.py — interactive drift calibration for /cmd_vel
# Creates cmdvel_drifts.py with DriftLeft/DriftRight(seconds) using your chosen params.

import time, sys
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist

RATE_HZ   = 20          # publish rate while holding a command
TEST_SEC  = 1.2         # duration per trial
GAP       = 0.35        # pause between trials
BASE_VY   = 0.20        # starting strafe speed (m/s)
BASE_WZ   = 0.50        # starting yaw speed (rad/s)

_node = None
_pub  = None

def _ensure():
    global _node, _pub
    if not rclpy.ok(): rclpy.init(args=None)
    if _node is None:
        _node = Node('drift_calib_cmdvel')
        _pub  = _node.create_publisher(Twist, '/cmd_vel', 10)

def _send(vx=0.0, vy=0.0, wz=0.0):
    _ensure()
    m = Twist(); m.linear.x=float(vx); m.linear.y=float(vy); m.angular.z=float(wz)
    _pub.publish(m)

def _hold(seconds, vx=0.0, vy=0.0, wz=0.0, rate=RATE_HZ):
    end = time.time() + float(seconds); dt = 1.0/float(rate)
    while time.time() < end:
        _send(vx,vy,wz); time.sleep(dt)
    _send(0,0,0); time.sleep(GAP)

def ask(msg, default=""):
    s = input(msg+" ").strip().lower()
    return s if s else default

def detect_left_signs():
    print("\\nStep 1 — Find a combo that clearly drifts LEFT.")
    trials = [
        ("A", +BASE_VY, +BASE_WZ),
        ("B", -BASE_VY, +BASE_WZ),
        ("C", +BASE_VY, -BASE_WZ),
        ("D", -BASE_VY, -BASE_WZ),
    ]
    for lab, vy, wz in trials:
        print(f" Trial {lab}: vy={vy:+.2f} m/s, wz={wz:+.2f} rad/s")
        _hold(TEST_SEC, vy=vy, wz=wz)
        ans = ask("Did that arc LEFT? [y/n]", "n")
        if ans == "y":
            return (vy, wz)
    print("No trial arced LEFT. Try raising BASE_VY or BASE_WZ at the top and rerun.")
    sys.exit(1)

def tune_curvature(vy, wz, label="LEFT"):
    print(f"\\nStep 2 — Tune {label} curvature. Pick one that looks best.")
    ks = [0.8, 1.0, 1.3]
    best = (wz, "B")
    for k, lab in zip(ks, ["A","B","C"]):
        wzk = wz * k
        print(f" Trial {lab}: vy={vy:+.2f}, wz={wzk:+.2f}")
        _hold(TEST_SEC, vy=vy, wz=wzk)
        score = ask(" Rate: 0=straight, 1=ok, 2=too tight  [0/1/2] ", "1")
        if score == "1": best = (wzk, lab)
    print(f" Chosen wz={best[0]:+.2f} (from {best[1]})")
    return vy, best[0]

def main():
    print("=== /cmd_vel Drift Calibrator ===")
    print("Make sure the mecanum controller is running and /cmd_vel has a subscriber.")
    # LEFT side
    vyL, wzL = detect_left_signs()
    vyL, wzL = tune_curvature(vyL, wzL, "LEFT")

    # RIGHT: mirror vy, flip yaw (usually), then tune
    vyR, wzR = -vyL, -wzL
    print("\\nStep 3 — Quick right mirror trial")
    print(f" Trial R0: vy={vyR:+.2f}, wz={wzR:+.2f}")
    _hold(TEST_SEC, vy=vyR, wz=wzR)
    ok = ask("Did that arc RIGHT? [y/n]", "y")
    if ok != "y":
        print(" We'll tune RIGHT separately.")
        vyR, wzR = detect_left_signs()  # reuse finder but you'll say 'y' when it arcs LEFT; we then flip signs
        vyR, wzR = -vyR, -wzR
    vyR, wzR = tune_curvature(vyR, wzR, "RIGHT")

    # Save file
    out = f"""
# cmdvel_drifts.py — generated by calibrate_drift_cmdvel.py
# Simple seconds-only API for class use: DriftLeft(seconds), DriftRight(seconds), Stop()
import time, atexit, rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist

RATE_HZ = {RATE_HZ}
GAP     = {GAP:.2f}
VY_LEFT  = {vyL:.3f}
WZ_LEFT  = {wzL:.3f}
VY_RIGHT = {vyR:.3f}
WZ_RIGHT = {wzR:.3f}

_node=None; _pub=None
def _ensure():
    global _node,_pub
    if not rclpy.ok(): rclpy.init(args=None)
    if _node is None:
        _node = Node('cmdvel_drifts')
        _pub  = _node.create_publisher(Twist, '/cmd_vel', 10)

def _send(vx=0.0, vy=0.0, wz=0.0):
    _ensure()
    m=Twist(); m.linear.x=float(vx); m.linear.y=float(vy); m.angular.z=float(wz)
    _pub.publish(m)

def _hold(seconds, vy=0.0, wz=0.0, rate=RATE_HZ):
    end=time.time()+float(seconds); dt=1.0/float(rate)
    while time.time()<end:
        _send(0.0, vy, wz)
        time.sleep(dt)
    _send(0.0,0.0,0.0); time.sleep(GAP)

def DriftLeft(seconds=1.2):  _hold(seconds, vy=VY_LEFT,  wz=WZ_LEFT)
def DriftRight(seconds=1.2): _hold(seconds, vy=VY_RIGHT, wz=WZ_RIGHT)
def Stop(): _send(0,0,0)

atexit.register(Stop)
"""
    open("cmdvel_drifts.py","w").write(out)
    print("\\nSaved cmdvel_drifts.py with your drift settings.")
    print("Use it in a notebook cell like:")
    print("  import importlib, cmdvel_drifts as cd; importlib.reload(cd)")
    print("  cd.DriftLeft(1.2); cd.DriftRight(1.2); cd.Stop()")

if __name__ == "__main__":
    try:
        main()
    finally:
        try: _send(0,0,0)
        except: pass
'''
Path("calibrate_drift_cmdvel.py").write_text(cal_py.strip()+"\n")
print("Wrote calibrate_drift_cmdvel.py")


Wrote calibrate_drift_cmdvel.py


In [2]:
%run calibrate_drift_cmdvel.py


=== /cmd_vel Drift Calibrator ===
Make sure the mecanum controller is running and /cmd_vel has a subscriber.
\nStep 1 — Find a combo that clearly drifts LEFT.
 Trial A: vy=+0.20 m/s, wz=+0.50 rad/s


Did that arc LEFT? [y/n]  y


\nStep 2 — Tune LEFT curvature. Pick one that looks best.
 Trial A: vy=+0.20, wz=+0.40


 Rate: 0=straight, 1=ok, 2=too tight  [0/1/2]   1


 Trial B: vy=+0.20, wz=+0.50


...

 Trial B: vy=-0.20, wz=-0.65


 Rate: 0=straight, 1=ok, 2=too tight  [0/1/2]   1


 Trial C: vy=-0.20, wz=-0.85


 Rate: 0=straight, 1=ok, 2=too tight  [0/1/2]   1


 Chosen wz=-0.85 (from C)
\nSaved cmdvel_drifts.py with your drift settings.
Use it in a notebook cell like:
  import importlib, cmdvel_drifts as cd; importlib.reload(cd)
  cd.DriftLeft(1.2); cd.DriftRight(1.2); cd.Stop()
